In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
data=pd.read_csv(r'D:\BaiduNetdiskDownload\effect_tb.csv',header=None)
data.columns=['dt','user_id','label','dmp_id']
#日志天数属性用不上删除该列
data=data.drop(columns='dt')
data.head(3)

,user_id,label,dmp_id
0,1,0,1
1,1000004,0,1
2,1000004,0,2


In [2]:
data.describe()

,user_id,label,dmp_id
count,2.645958e+06,2.645958e+06,2.645958e+06
mean,3.112995e+06,1.456297e-02,1.395761e+00
std,1.828262e+06,1.197952e-01,6.920480e-01
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,1.526772e+06,0.000000e+00,1.000000e+00
50%,3.062184e+06,0.000000e+00,1.000000e+00
75%,4.721132e+06,0.000000e+00,2.000000e+00
max,6.265402e+06,1.000000e+00,3.000000e+00


In [ ]:
2.重复值处理

In [3]:
data.shape

(2645958, 3)

In [4]:
data.nunique()

user_id    2410683
label            2
dmp_id           3
dtype: int64

In [5]:
#删除重复值
data=data.drop_duplicates()

In [ ]:
3.空值处理

In [6]:
#检查空值个数
data.isnull().sum()

user_id    0
label      0
dmp_id     0
dtype: int64

In [ ]:
数据集无空值，无需进行处理

In [ ]:
4.异常值处理

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2632975 entries, 0 to 2645957
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   label    int64
 2   dmp_id   int64
dtypes: int64(3)
memory usage: 80.4 MB


In [8]:
data_pivot=data.pivot_table(index='dmp_id',columns='label',values='user_id',aggfunc='count',margins=True)

label,0,1,All
dmp_id,,,
1,1881745,23918,1905663
2,404811,6296,411107
3,307923,8282,316205
All,2594479,38496,2632975


In [ ]:
dmp_id=1、dmp_id=2 和 dmp_id=3 ：未点击广告的用户数远大于点击广告的用户数。合理
dmp_id=2 和 dmp_id=3 ： 两种策略所得到点击广告用户数没有显著差别。合理
未发现异常取值，无需处理


In [12]:
data.dtypes

user_id    int64
label      int64
dmp_id     int64
dtype: object

In [ ]:
数据类型正常，无需处理

In [ ]:
2.2样本容量检验
在进行A/B测试前，需检查样本容量是否满足试验所需最小值。
首先需要设定点击率基准线以及最小提升比例，我们将对照组的点击率设为基准线，从前述可知对照组点击率为1.26%
假设我们希望新的营销策略能让广告点击率至少提升1个百分点，则每组所需最小样本量为2658

In [24]:
from statsmodels.stats.power import zt_ind_solve_power
from statsmodels.stats.proportion import proportion_effectsize

# 输入参数
baseline_rate = 0.0126       # 基础转化率（对照组点击率）
min_effect = 0.01           # 最小期望提升（绝对提升
alpha = 0.05                # 显著性水平
power = 0.8    # 统计功效（1 - 第二类错误率） 0.7155得2167

# 计算效应量（effect size）
effect_size = proportion_effectsize(prop1=baseline_rate + min_effect, prop2=baseline_rate)

# 计算最小样本量（每组）
sample_size = zt_ind_solve_power(
    effect_size=effect_size,
    nobs1=None,
    alpha=alpha,
    power=power,
    alternative='two-sided'
)

print(f"每组所需最小样本量: {int(sample_size)}")

每组所需最小样本量: 2658


In [25]:
#检查样本量
data['dmp_id'].value_counts()

dmp_id
1    1905663
2     411107
3     316205
Name: count, dtype: int64

In [ ]:
两组营销活动的样本量分别为41.11万和31.62万，满足最小样本量需求。

In [35]:
#计算各组点击率
print('对照组：',data[data['dmp_id']==1]['label'].mean())
print('营销策略一：',data[data['dmp_id']==2]['label'].mean())
print('营销策略二：',data[data['dmp_id']==3]['label'].mean())

对照组： 0.012551012429794775
营销策略一： 0.015314747742072015
营销策略二： 0.026191869198779274


In [ ]:
可以看到策略一和策略二相较对照组在点击率上都有不同程度提升。其中策略一提升0.2个百分点，策略二提升1.3个百分点，只有策略二满足了前面我们对点击率提升最小值的要求。
接下来需要进行假设检验，看策略二点击率的提升是否显著。
a.原假设和备择假设
记对照组点击率为p1，策略二点击率为p2，则:
原假设 H0:p1≥p2
备择假设 H1: p1 < p2
b.分布类型、检验类型和显著性水平
样本服从二点分布(伯努利分布)，独立双样本，样本大小n>30，总体均值和标准差未知，所以采用Z检验。显著性水平α取
0.05。

In [37]:
#用python statsmodels包计算z值和p值
#首先计算对照组和营销策略二的用户数和点击数

#用户数
n_old=len(data[data.dmp_id==1])
n_new=len(data[data.dmp_id==3])

#点击数
c_old=len(data[data.dmp_id==1][data.label==1])
c_new=len(data[data.dmp_id==3][data.label==1])


import statsmodels.stats.proportion as sp
z_score,p=sp.proportions_ztest([c_old,c_new],[n_old,n_new],alternative='two-sided')
print('检验统计量z:',z_score,'p值：',p)

检验统计量z: -59.44168632985996 p值： 0.0


C:\Users\86178\AppData\Local\Temp\ipykernel_172\2410972183.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c_old=len(data[data.dmp_id==1][data.label==1])
C:\Users\86178\AppData\Local\Temp\ipykernel_172\2410972183.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c_new=len(data[data.dmp_id==3][data.label==1])


In [ ]:
p值约等于0，p<α，故拒绝原假设，策略二的点击率提升显著


In [39]:
#作为补充，我们再检验下策略一的点击率提升是否显著
#计算策略一的用户数、点击数
n_new2=len(data[data.dmp_id==2])
c_new2=len(data[data.dmp_id==2][data.label==1])
z_score,p=sp.proportions_ztest([c_old,c_new2],[n_old,n_new2],alternative='two-sided')
print('策略一检验统计量z:',z_score,'策略一p值：',p)

策略一检验统计量z: -14.165873564308429 策略一p值： 1.4900243485475164e-45


C:\Users\86178\AppData\Local\Temp\ipykernel_172\105735733.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c_new2=len(data[data.dmp_id==2][data.label==1])


In [ ]:
p值约等于1.49e-45，p<α，但因为前面我们设置了对点击率提升的最小值要求，这里仍只选择第二组策略进行推广

In [ ]:
4.结论
综上所述，两种营销策略中，策略二对广告点击率有显著提升效果，且相较于对照组点击率提升了近一倍，因而在两组营销策略中应选择第二组进行推广。